In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("./DataBaseFINAL.csv")
df=df.dropna(axis=0)
df.head()

,category,description,description_process,id,input,output
0,ad-hoc,"[<p><span class=""tex-font-style-underline"">Pet...",petya loves lucky numbers very much everybody ...,0,"[<div class=""input-specification""><div class=""...","[<div class=""output-specification""><div class=..."
1,ad-hoc,[<p>Walking through the streets of Marshmallow...,walking through the streets of marshmallow cit...,0,"[<div class=""input-specification""><div class=""...","[<div class=""output-specification""><div class=..."
2,ad-hoc,"[<p>Two <span class=""tex-font-style-bf"">positi...",two positive integers a and b have a sum of s ...,0,"[<div class=""input-specification""><div class=""...","[<div class=""output-specification""><div class=..."
3,ad-hoc,[<p>Each month Blake gets the report containin...,each month blake gets the report containing ma...,0,"[<div class=""input-specification""><div class=""...","[<div class=""output-specification""><div class=..."
4,ad-hoc,[<p>Andrew and Jerry are playing a game with H...,andrew and jerry are playing a game with harry...,0,"[<div class=""input-specification""><div class=""...","[<div class=""output-specification""><div class=..."


In [3]:
contador=0
lista=[]
while(contador<len(df['category'])):
    if(contador==3772 or contador==3815):
        contador+=2
    if(contador==3785 or contador==4683 or contador==5046):
        contador+=1
    if(df['category'][contador]=='ad-hoc'):
        lista+=[contador]
    contador+=1
df=df.drop(lista)

In [4]:
import spacy
nlp = spacy.load('en')

In [5]:
master=[]
for elemento in df['description_process']:
    doc=nlp(elemento)
    lista=[]
    for token in doc:
        if(token.text not in nlp.Defaults.stop_words and token.lemma_ not in nlp.Defaults.stop_words and token.text.isalpha()):
            lista+=[token.lemma_]
    master+=[' '.join(lista)]

In [6]:
df['tokens']=master

In [7]:
X = df['tokens'].values
y = df['category'].values

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [9]:
from sklearn.preprocessing import LabelEncoder
X = df['tokens'].values
y = df['category'].values
encoder = LabelEncoder()
encoder.fit(y)

LabelEncoder()

In [10]:
from sklearn.model_selection import StratifiedKFold
vetor_X_train=[]
vetor_y_train=[]
vetor_X_test=[]
vetor_y_test=[]
split = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
for train_index, val_index in split.split(X, y):
    vetor_X_train+=[X[train_index]]
    vetor_X_test+=[X[val_index]]
    vetor_y_train+=[y[train_index]]
    vetor_y_test+=[y[val_index]]

In [11]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer                    
from keras.preprocessing.sequence import pad_sequences
from keras import layers

In [12]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)

vocab_size = len(tokenizer.word_index) + 1   
maxlen = 300
embedding_dim = 100
vocab_size
idx_word=tokenizer.index_word

In [ ]:
num_words=vocab_size
word_idx=tokenizer.word_index

import numpy as np
glove_vectors = './glove.6B.50d.txt'
glove = np.loadtxt(glove_vectors, dtype='str', comments=None)

# Extract the vectors and words
vectors = glove[:, 1:].astype('float')
words = glove[:, 0]

# Create lookup of words to vectors
word_lookup = {word: vector for word, vector in zip(words, vectors)}

# New matrix to hold word embeddings
embedding_matrix = np.zeros((num_words, vectors.shape[1]))

for i, word in enumerate(word_idx.keys()):
    # Look up the word embedding
    vector = word_lookup.get(word, None)

    # Record in matrix
    if vector is not None:
        embedding_matrix[i + 1, :] = vector

In [ ]:
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding
def baseline_model():
    model = Sequential()

    # Embedding layer
    model.add(
        Embedding(input_dim=vocab_size,
                  input_length = 300,
                  output_dim=50,
                  weights=[embedding_matrix],
                  trainable=False,
                  mask_zero=True))

    # Masking layer for pre-trained embeddings
    model.add(Masking(mask_value=0.0))

    # Recurrent layer
    model.add(LSTM(64, return_sequences=False, 
                   dropout=0.1, recurrent_dropout=0.1))
    
     # Recurrent layer
    model.add(LSTM(64, return_sequences=False, 
                   dropout=0.1, recurrent_dropout=0.1))
    
     # Recurrent layer
    model.add(LSTM(64, return_sequences=False, 
                   dropout=0.1, recurrent_dropout=0.1))
    
     # Recurrent layer
    model.add(LSTM(64, return_sequences=False, 
                   dropout=0.1, recurrent_dropout=0.1))
    
    

    # Fully connected layer
    model.add(layers.Dense(64, activation='relu'))
    model.add(Dropout(p=0.1))
    model.add(layers.Dense(7, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
contador=0
predictions=[]
while(contador<len(vetor_X_train)):
    X_train=vetor_X_train[contador]
    y_train=vetor_y_train[contador]
    X_test=vetor_X_test[contador]
    y_test=vetor_y_test[contador]
    
    encoded_y = encoder.transform(y_train)
    # convert integers to dummy variables (i.e. one hot encoded)
    y_train = np_utils.to_categorical(encoded_y)
        
#     X_train = vectorizer.fit_transform(X_train)
    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)
    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
    
    estimator = KerasClassifier(build_fn=baseline_model, epochs=30, batch_size=20)
    estimator.fit(X_train, y_train)
#     y_pred = estimator.predict(vectorizer.fit_transform(X_test))
    y_pred = estimator.predict(X_test)
    predictions+=[y_pred]

    contador+=1

In [ ]:
y_pred=[]
for lista1 in predictions:
    for elemento1 in encoder.inverse_transform(lista1):
        y_pred+=[elemento1]


In [ ]:
y_test_sla=[]
for lista2 in vetor_y_test:
    for elemento2 in lista2:
        y_test_sla+=[elemento2]

In [ ]:
from sklearn import metrics

In [ ]:
print(metrics.confusion_matrix(y_test_sla,y_pred))

In [ ]:
print(metrics.classification_report(y_test_sla,y_pred))

In [ ]:
print(metrics.accuracy_score(y_test_sla,y_pred))

In [ ]:
np.savetxt('ytrue_RNN', y_test_sla, fmt='%s')
np.savetxt('ypred_RNN', y_pred, fmt='%s')